In [1]:
!pip install geopandas shapely tqdm

In [37]:
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np

# Load shp file
gdf_origin = gpd.read_file("E:/Users/Administrator/Downloads/Countries_cope/Countries_cope/Countries_cope/Africa_8/Uganda/Uganda_pop.shp")
gdf_dest = gpd.read_file("E:/Users/Administrator/Downloads/Countries_cope/Countries_cope/Countries_cope/Africa_8/Uganda/Uganda_vac.shp")

In [38]:
# Convert CRS to UTM projected coordinates (select according to region)
gdf_origin = gdf_origin.to_crs("EPSG:32635")
gdf_dest = gdf_dest.to_crs("EPSG:32635")

In [39]:
# 对于 gdf_origin GeoDataFrame
origin_points = gdf_origin.geometry.apply(lambda geom: (geom.x, geom.y) if geom is not None else (None, None)).tolist()

# 对于 gdf_dest GeoDataFrame
dest_points = gdf_dest.geometry.apply(lambda geom: (geom.x, geom.y) if geom is not None else (None, None)).tolist()

In [40]:
gdf_origin

,pointid,grid_code,geometry
0,1,11.6870,POINT (1274128.557 466762.224)
1,3,17.7253,POINT (1274784.037 466396.865)
2,4,17.0864,POINT (1274971.201 466305.732)
3,5,15.2194,POINT (1275159.194 466121.803)
4,8,15.4325,POINT (1275441.184 465845.908)
...,...,...,...
1265112,4026302,15.9101,POINT (824957.277 -163337.673)
1265113,4026305,11.0994,POINT (824678.697 -163429.537)
1265114,4026307,15.9457,POINT (824864.336 -163429.781)
1265115,4026309,11.9666,POINT (825049.975 -163430.025)


In [41]:
gdf_dest

,Vaccinatio,Address,Latitude,Longitude,geometry
0,Abarilela Health Centre III,Uganda,1.962000,33.818200,POINT (1260057.028 218414.692)
1,Abim hospital,"MMX5+H3H, Abim, Uganda",2.698932,33.657724,POINT (1241696.540 300351.428)
2,Adjumani Hospital,"9QFV+M2X, Adjumani, Uganda",3.374242,31.792596,POINT (1032999.337 374273.778)
3,Aduku Health Centre IV,"Aduku, Uganda",2.018354,32.720961,POINT (1137271.733 224213.101)
4,Agwata Health Centre III,"X2JX+8PC, Adwoki, Uganda",1.980814,33.049267,POINT (1173991.268 220173.599)
...,...,...,...,...,...
469,Wakiso Health Centre IV,"Wakiso Town Council,, Namirembe Rd, Wakiso, Ug...",0.396518,32.477291,POINT (1110407.511 44029.490)
470,Wera Health Centre III,"VQ83+G8H, Wera, Uganda",1.866324,33.753358,POINT (1252836.980 207735.689)
471,Yinga Health Centre III,Uganda,3.260600,31.242000,POINT (971700.589 361392.728)
472,Yumbe Health Centre IV,Uganda,3.477000,31.197000,POINT (966584.251 385355.526)


In [ ]:
from tqdm import tqdm
from shapely.geometry import Point
from scipy.spatial import cKDTree
import geopandas as gpd

# 使用目标点构建 KD 树
tree = cKDTree(dest_points)

# 对每个源点查询最近的k个目标点的距离和索引
# 修改这个k的数量
distances, indices = tree.query(origin_points, k=474)

data = []

for i in tqdm(range(len(origin_points))):
    origin = gdf_origin.iloc[i]  
    origin_point = Point(origin_points[i])  
    for j in range(474):  # 修改这里的数量 是dest的数量
        dest_idx = indices[i][j]  
        dest_point = Point(dest_points[dest_idx]) 
        distance = distances[i][j] / 1000  # 将距离转换为公里
        # 将源点、目标点、距离、pointid 和 grid_code 添加到数据列表中
        data.append({
            'origin': origin_point,
            'dest': dest_point,
            'distance': distance,
            'pointid': origin['pointid'],
            'grid_code': origin['grid_code']
        })
gdf_result = gpd.GeoDataFrame(data, geometry='dest')

# 设置 GeoDataFrame 的 CRS 这里也需要更改
gdf_result.set_crs("EPSG:32635", inplace=True)


  2%|█▎                                                                     | 23863/1265117 [03:34<3:01:20, 114.08it/s]

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

gdf_result['origin'] = gdf_result['origin'].astype(str)

def filter_rows(group):
    # 判断组内所有distance是否都大于30
    if (group['distance'] > 30).all():
        # 再判断组内所有distance是否都大于50
        if (group['distance'] > 50).all():
            # 如果是，则保留最小的一个distance
            return group.nsmallest(1, 'distance')
        else:
            # 否则，保留所有小于等于50的distance，并从这些中保留最小的三个
            return group[group['distance'] <= 50].nsmallest(3, 'distance')
    else:
        # 否则，保留所有小于等于30的distance
        return group[group['distance'] <= 30]

tqdm.pandas(desc="Processing groups")
result_new = gdf_result.groupby('origin', as_index=False, group_keys=False).progress_apply(filter_rows)

In [ ]:
result_new

In [ ]:
import geopandas as gpd
from shapely.wkt import loads
if isinstance(result_new['origin'].iloc[0], str):
    result_new['origin'] = result_new['origin'].apply(loads)
if isinstance(result_new['dest'].iloc[0], str):
    result_new['dest'] = result_new['dest'].apply(loads)

gdf_result = gpd.GeoDataFrame(result_new, geometry='origin', crs="EPSG:32635")
gdf_result.set_geometry('origin', inplace=True)
gdf_result['origin'] = gdf_result['origin'].to_crs("EPSG:4326")

gdf_result['dest'] = gpd.GeoSeries(gdf_result['dest'], crs="EPSG:32635")
gdf_result['dest'] = gdf_result['dest'].to_crs("EPSG:4326")

In [ ]:
gdf_result.to_csv("E:/Users/Administrator/Downloads/Countries_KDTree/Africa/Uganda.csv")

In [32]:
gdf_result

,origin,dest,distance,pointid,grid_code
3542740,POINT (31.80333 -20.88917),POINT (30.06725 -20.30962),192.343387,2955190,10.3864
3526960,POINT (31.80000 -20.78250),POINT (30.06725 -20.30962),188.446427,2895908,10.7473
154480,POINT (31.69167 -16.93917),POINT (31.09370 -17.74865),110.096665,286175,12.9891
154240,POINT (31.69167 -16.93833),POINT (31.09370 -17.74865),110.172153,285702,12.0188
3566820,POINT (31.80667 -20.98583),POINT (30.06725 -20.30962),196.454604,3005597,16.1243
...,...,...,...,...,...
154220,POINT (31.69083 -16.93833),POINT (31.09370 -17.74865),110.120785,285701,11.3208
3530400,POINT (31.80000 -20.80583),POINT (30.06725 -20.30962),189.169614,2908375,10.2113
3566800,POINT (31.80583 -20.98583),POINT (30.06725 -20.30962),196.374111,3005596,11.0588
2654940,POINT (31.75083 -19.16083),POINT (31.19967 -17.84341),157.378185,1879446,10.5620
